In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("Dizex/FoodBaseBERT")
model = AutoModelForTokenClassification.from_pretrained("Dizex/FoodBaseBERT")
pipe = pipeline("ner", model=model, tokenizer=tokenizer)

/Users/jacobchang/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/jacobchang/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
example = "Today's meal: Fresh olive poke bowl topped with chia seeds. Very delicious!"

ner_entity_results = pipe(example)
print(ner_entity_results)

Device set to use mps:0


[{'entity': 'B-FOOD', 'score': np.float32(0.6692053), 'index': 6, 'word': 'Fresh', 'start': 14, 'end': 19}, {'entity': 'I-FOOD', 'score': np.float32(0.5334641), 'index': 7, 'word': 'olive', 'start': 20, 'end': 25}, {'entity': 'I-FOOD', 'score': np.float32(0.9861605), 'index': 8, 'word': 'p', 'start': 26, 'end': 27}, {'entity': 'I-FOOD', 'score': np.float32(0.9927375), 'index': 9, 'word': '##oke', 'start': 27, 'end': 30}, {'entity': 'I-FOOD', 'score': np.float32(0.97065824), 'index': 10, 'word': 'bowl', 'start': 31, 'end': 35}, {'entity': 'B-FOOD', 'score': np.float32(0.9957469), 'index': 13, 'word': 'ch', 'start': 48, 'end': 50}, {'entity': 'B-FOOD', 'score': np.float32(0.97199684), 'index': 14, 'word': '##ia', 'start': 50, 'end': 52}, {'entity': 'I-FOOD', 'score': np.float32(0.99487525), 'index': 15, 'word': 'seeds', 'start': 53, 'end': 58}]


In [5]:
foods = []
current_food = ""

for ent in ner_entity_results:
    word = ent["word"]
    # Handle wordpieces (like ##oke -> oke)
    if word.startswith("##"):
        word = word[2:]
    
    if ent["entity"].startswith("B-FOOD"):
        if current_food:
            foods.append(current_food.strip())
        current_food = word + " "
    elif ent["entity"].startswith("I-FOOD"):
        current_food += word + " "

# Add last entity
if current_food:
    foods.append(current_food.strip())

# Deduplicate and clean
foods = [f.replace(" ##", "").replace("  ", " ").strip() for f in foods]
foods = list(dict.fromkeys(foods))  # preserve order

# Store in a dictionary
food_dict = {"foods": foods}
print(food_dict)

{'foods': ['Fresh olive p oke bowl', 'ch', 'ia seeds']}
